In [4]:
import pandas as pd
import numpy as np


df = pd.read_csv("data/train_lenta.csv")
df.shape

(714510, 195)

In [3]:
df.head()

,age,cheque_count_12m_g20,cheque_count_12m_g21,cheque_count_12m_g25,cheque_count_12m_g32,cheque_count_12m_g33,cheque_count_12m_g38,cheque_count_12m_g39,cheque_count_12m_g41,cheque_count_12m_g42,...,sale_sum_6m_g26,sale_sum_6m_g32,sale_sum_6m_g33,sale_sum_6m_g44,sale_sum_6m_g54,stdev_days_between_visits_15d,stdev_discount_depth_15d,stdev_discount_depth_1m,target,treatment
0,81.0,9.0,14.0,12.0,7.0,19.0,8.0,4.0,18.0,9.0,...,0.00,314.91,1045.57,134.36,0.00,0.0,0.0,0.0000,0,1
1,56.0,17.0,13.0,24.0,14.0,3.0,2.0,2.0,13.0,13.0,...,582.83,3489.72,119.52,1031.92,218.12,0.0,NaN,0.1608,0,0
2,63.0,0.0,0.0,0.0,0.0,9.0,2.0,8.0,0.0,2.0,...,93.67,0.00,424.98,0.00,0.00,0.0,0.0,0.0000,0,1
3,56.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,4.0,1.0,...,34.89,0.00,109.90,81.99,77.38,0.0,0.0,0.0000,0,0
4,47.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,2.0,0.0,...,0.00,0.00,209.14,241.50,89.99,0.0,0.0,0.0000,0,1
